In [76]:
from ezc3d import c3d
from pyomeca import Markers, Analogs
import config as cfg
import importlib

def normalize_emg(emg):
    emg_p = (
    emg.meca.band_pass(order=2, cutoff=[10, 425])
    .meca.center()
    .meca.abs()
    .meca.low_pass(order=4, cutoff=5, freq=emg.rate)
    .meca.normalize(ref=None, scale=1)
    )
    return emg_p

def name_body(data_path, marker_list):
    s = data_path.split('\\')[-1]
    body = s.split('-')[3]+":"
    for i in range(len(marker_list)):
        marker_list[i] = body+marker_list[i]
    return marker_list

def data_markers(data_path, marker_list):
    frame_rate = 450
    for i in range(len(marker_list)):
        if i == 0:
            data_markers = Markers.from_c3d(data_path, usecols=[marker_list[0]])
            data_markers = data_markers.meca.time_normalize(n_frames=frame_rate)
            data_markers = data_markers.meca.to_wide_dataframe()
        else:
            tmp_markers = Markers.from_c3d(data_path, usecols=[marker_list[i]])
            tmp_markers = tmp_markers.meca.time_normalize(n_frames=frame_rate)
            tmp_markers = tmp_markers.meca.to_wide_dataframe()
            data_markers = data_markers.join(tmp_markers)      
    cols = [c for c in data_markers.columns if c.lower()[:4] != 'ones']
    markers_dataframe = data_markers[cols]
    return markers_dataframe

def dataframe(data_path):
    mark_channels = name_body(data_path,cfg.marker_channels[:13])
    plat_channels = name_body(data_path,cfg.platform_channels)
    ang_channels = name_body(data_path,cfg.angles_channels)
    forc_channels = name_body(data_path,cfg.forces_channels)
    mome_channels = name_body(data_path,cfg.moments_channels)
    powe_channels = name_body(data_path,cfg.powers_channels)
    
    full_data = data_markers(data_path,mark_channels)
    full_data = full_data.join(data_markers(data_path,cfg.marker_channels[13:]))
    emg1 = Analogs.from_c3d(data_path, usecols=cfg.emg_channels[:9])
    emg2 = Analogs.from_c3d(data_path, usecols=cfg.emg_channels[9:])
    norm_emg1 = normalize_emg(emg1)
    norm_emg2 = normalize_emg(emg2)
    norm_emg1 = norm_emg1.meca.to_wide_dataframe()
    norm_emg2 = norm_emg2.meca.to_wide_dataframe()
    
    full_data = full_data.join(norm_emg1)
    full_data = full_data.join(norm_emg2)
    
    full_data = full_data.join(data_markers(data_path,plat_channels))
    full_data = full_data.join(data_markers(data_path,ang_channels))
    full_data = full_data.join(data_markers(data_path,forc_channels))    
    full_data = full_data.join(data_markers(data_path,mome_channels))
    
    data_powers_tmp = data_markers(data_path,powe_channels)
    cols = [c for c in data_powers_tmp.columns if c.lower()[:1] == 'z']
    data_powers = data_powers_tmp[cols]
    
    full_data = full_data.join(data_powers)
    
    return full_data

# Wektor X

In [79]:
importlib.reload(cfg)
data_path = r"Z:\baza\2020-08-05-B0444-S01-E01-T01-K05.c3d"
n = dataframe(data_path)

In [80]:
n

,x_B0444:LSHO,y_B0444:LSHO,z_B0444:LSHO,x_B0444:LELB,y_B0444:LELB,z_B0444:LELB,x_B0444:LWRA,y_B0444:LWRA,z_B0444:LWRA,x_B0444:RSHO,...,z_B0444:LAnklePower,z_B0444:RHipPower,z_B0444:RKneePower,z_B0444:RAnklePower,z_B0444:LShoulderPower,z_B0444:LElbowPower,z_B0444:LWristPower,z_B0444:RShoulderPower,z_B0444:RElbowPower,z_B0444:RWristPower
time,,,,,,,,,,,,,,,,,,,,,
0.000000,-430.316956,-185.981323,1455.084595,-420.517090,-10.783237,1158.911133,-240.802902,-64.469841,1310.695801,-166.688095,...,-0.001600,0.005166,0.129711,0.011504,-0.000042,-0.001521,0.000055,-0.004303,-0.000009,0.000030
0.007862,-430.230307,-185.036784,1455.081519,-420.659061,-9.915171,1158.827342,-240.697290,-63.485691,1310.321160,-166.660958,...,-0.001251,0.003192,0.125569,0.001135,-0.000243,-0.001710,0.000048,-0.005325,-0.000012,0.000034
0.015724,-430.145514,-184.047773,1454.999006,-420.769443,-9.073192,1158.615640,-240.572768,-62.455021,1309.861654,-166.623246,...,-0.000909,0.000641,0.119367,-0.008635,-0.000426,-0.001922,0.000033,-0.006468,-0.000015,0.000041
0.023586,-430.065960,-183.008092,1454.833916,-420.845254,-8.266465,1158.265691,-240.428356,-61.369117,1309.318614,-166.577019,...,-0.000570,-0.002391,0.112010,-0.018133,-0.000594,-0.002161,0.000014,-0.007757,-0.000019,0.000051
0.031448,-429.994182,-181.919643,1454.598587,-420.889915,-7.496638,1157.793051,-240.266194,-60.228705,1308.707315,-166.526346,...,-0.000233,-0.005750,0.106235,-0.026686,-0.000795,-0.002431,-0.000006,-0.009179,-0.000023,0.000064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3.498552,-466.175040,-166.399097,1439.481604,-460.220714,-2.576048,1137.535410,-271.755300,-38.674895,1285.975952,-194.224323,...,0.022464,-0.039225,0.267892,-0.108602,-0.011709,-0.003887,0.000033,-0.024604,0.000323,0.000058
3.506414,-467.589393,-162.947210,1438.648001,-462.520435,0.640902,1136.485706,-273.494249,-34.727022,1284.377769,-195.450888,...,0.022585,-0.037740,0.298151,-0.110792,-0.011191,-0.003945,0.000031,-0.026878,0.000364,0.000038
3.514276,-469.088094,-159.484517,1437.755883,-465.021723,3.903564,1135.396495,-275.399930,-30.647730,1282.707272,-196.685790,...,0.023058,-0.029988,0.324061,-0.112932,-0.010414,-0.004035,0.000024,-0.029264,0.000405,0.000011


# Wektor Y

In [163]:
import glob
from pathlib import Path
import config as cfg
import pandas as pd

In [175]:
importlib.reload(cfg)

s = data_path.split('\\')[-1]
body = s.split('-')[3]

i = 0
b = []
m = []
for path in Path(r'Z:\baza').rglob('*.c3d'):
    s = str(path).split('\\')[-1]
    body = s.split('-')[3]
    b.append(cfg.body.index(body))
    m.append((cfg.move).index(s.split('-')[5]))
    
    

In [176]:
d = {'osoba' : pd.Series(b),
     'ruch' : pd.Series(m)
        }

In [177]:
df = pd.DataFrame(d)

In [178]:
df

,osoba,ruch
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
3781,14,4
3782,14,4
3783,14,4
3784,14,4


In [189]:
x=pd.read_excel('adnotacja.xlsx', usecols='A:Y',skiprows=1, nrows=75 )

In [190]:
x

,Osoba,Ruch,wysokość jelca,pozycja sztychu,ustawienie stóp,łokcie blisko siebie,ruch noga do poz. końcowej,wyprostoawnie w łokciach,wyrzut rąk do przodu,uderzenie odpowiednim ostrzem,...,sztych ląduje po lewej,poniesienie jelca nad głowę,poziomy ruch ostrza na wysokości głowy,dokręcenie dłoni,skrzyżowanie rąk,odpowiednio mocne dokręcenie ostrza,trzymanie rąk po prawej stronie,ruch wędkowy,uderzenie po środku lini ciała,po uderzeniu tylnia noga i plecy są w jednej linii
0,B0444,E01,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,B0444,E02,1,1,1,1,1,1,1,1,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,B0444,E03,1,1,1,1,1,1,1,1,...,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,B0444,E04,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN
4,B0444,E05,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,B0458,E01,0,1,1,0,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,B0458,E02,0,1,1,1,1,1,1,0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,B0458,E03,1,1,1,0,1,1,1,0,...,NaN,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
73,B0458,E04,0,1,1,0,0,1,1,0,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN
